In [ ]:
%load_ext autoreload
%autoreload 2
from wind_processes import SetSequenceWindProcess
import numpy as np
import time
from tqdm import tqdm
from dynamic_windfarm_env import DynamicPriviegedWindFarmEnv

experiments = [
    {
        'name': '2wt_dynamic',
        'update_rule': 'observation_points',
        'turbine_layout': [[400, 1300], [500, 500]],
        'mast_layout': [[0, 0, 2000, 2000], [0, 2000, 0, 2000]],
        'wind_directions': np.ones(1000//5) * 270,
    },
    {
        'name': '3x3wt_dynamic',
        'update_rule': 'observation_points',
        'turbine_layout': [[600, 1500, 2400, 600, 1500, 2400, 600, 1500, 2400], [600, 600, 600, 1500, 1500, 1500, 2400, 2400, 2400]],
        'mast_layout': [[0, 0, 3000, 3000], [0, 3000, 0, 3000]],
        'wind_directions': np.concatenate((
            np.ones(300//5) * 255,
            np.linspace(255, 195, 300//5),
            np.ones(300//5) * 195,
        )),
    },
    {
        'name': '2wt',
        'update_rule': None,
        'turbine_layout': [[400, 1300], [500, 500]],
        'mast_layout': [[0, 0, 2000, 2000], [0, 2000, 0, 2000]],
        'wind_directions': np.ones(1000//5) * 270,
    },
    {
        'name': '3x3wt',
        'update_rule': None,
        'turbine_layout': [[600, 1500, 2400, 600, 1500, 2400, 600, 1500, 2400], [600, 600, 600, 1500, 1500, 1500, 2400, 2400, 2400]],
        'mast_layout': [[0, 0, 3000, 3000], [0, 3000, 0, 3000]],
        'wind_directions': np.concatenate((
            np.ones(300//5) * 255,
            np.linspace(255, 195, 300//5),
            np.ones(300//5) * 195,
        )),
    }
]

for experiment in experiments:
    turbine_layout = experiment['turbine_layout']
    mast_layout = experiment['mast_layout']
    wind_directions = experiment['wind_directions']
    update_rule = experiment['update_rule']
    name = experiment['name']
    
    EPISODES = 1
    EPISODE_LENGTH = len(wind_directions)

    env = DynamicPriviegedWindFarmEnv(
        turbine_layout=turbine_layout,
        #lidar_observations=('wind_speed', 'wind_direction'),
        mast_layout=mast_layout,
        floris="myfloris.json",
        episode_length=EPISODE_LENGTH,
        sorted_wind=False,
        wind_speed=None,
        changing_wind=False,
        action_representation='wind',
        observation_noise=0.0,
        verbose=False,
        update_rule=update_rule,
        load_pyglet_visualization=True,
        parallel_dynamic_computations=False,
        op_per_turbine=5,
        time_delta=5,
        op_wake_matrix_horizon=200,
    )
    env.seed(0)

    env.save_double_plot = True

    # Set the wind process
    wind_speeds = np.ones_like(wind_directions) * 8.0
    wind_process = SetSequenceWindProcess(wind_speeds, wind_directions)
    env.wind_process = wind_process

    env.reset()
    durations = []
    for i in range(EPISODES):
        for j in tqdm(range(EPISODE_LENGTH)):
            t0 = time.time()
            action = env.action_space.sample()*0
            obs, reward, t, tr, info = env.step(action)
            env.render()
            duration = time.time() - t0
            durations.append(duration/env.time_delta)
        env.reset()
    env.close()

    print(f"Episode durations: {np.mean(durations)}")

    # Plot the step and render times
    import matplotlib.pyplot as plt
    plt.plot(duration)
    plt.show()

In [ ]:
# This is for creating gifs from the saved images, move them to the right folder and run this to get the gifs
from PIL import Image

runs = [
    # "floris9", 
    # "floris2", 
    "dynamic9", 
    "dynamic2",
]
lens = [180, 200, 180, 200]
for run, length in zip(runs, lens):
    image_paths = [f"figures/renders/{run}/step_{i+1}.png" for i in range(length)]
    images = [Image.open(x) for x in image_paths]
    images[0].save(f'figures/renders/{run}_episode.gif',
                save_all=True, append_images=images[1:], optimize=False, duration=50, loop=0)
    print(f"Gif saved for {run}")
print("Gif saved")